In [4]:
#pip install pyreadr

In [2]:
import pandas as pd
import pyreadr
from functools import reduce

# Data Cleaning

## 0. Files for helping unify the USstate presentation

In [3]:
delineation_df = pd.read_excel('./data/delineation_file.xls')
delineation_df = delineation_df.iloc[1:1918]
delineation_df = delineation_df.rename(columns=delineation_df.iloc[0])
delineation_df = delineation_df.drop(1)
delineation_df['FIPS State Code'] = delineation_df['FIPS State Code'].astype(int)
delineation_df['CBSA Code'] = delineation_df['CBSA Code'].astype(int)
delineation_df = delineation_df[['CBSA Code', 'State Name', 'FIPS State Code', 'FIPS County Code']]

abbr_name = pd.read_csv('./data/abbr-name.csv')
abbr_name['abbr'] = abbr_name['abbr'].str.lower()

code_merged = pd.merge(delineation_df, abbr_name, left_on='State Name', right_on='full', how='left')
code_merged = code_merged.drop(columns=['full'])
code_merged.drop_duplicates()

,CBSA Code,State Name,FIPS State Code,FIPS County Code,abbr
0,10100,South Dakota,46,013,sd
1,10100,South Dakota,46,045,sd
2,10140,Washington,53,027,wa
3,10180,Texas,48,059,tx
4,10180,Texas,48,253,tx
...,...,...,...,...,...
1911,49700,California,6,101,ca
1912,49700,California,6,115,ca
1913,49740,Arizona,4,027,az
1914,49780,Ohio,39,119,oh


In [4]:
state = abbr_name['abbr'].to_list()

## 1. Crime Data 2011-2020

In [5]:
crime_2011 = pd.read_csv('./data/crime/crime_2011.csv')
crime_2012 = pd.read_csv('./data/crime/crime_2012.csv')
crime_2013 = pd.read_csv('./data/crime/crime_2013.csv')
crime_2014 = pd.read_csv('./data/crime/crime_2014.csv')
crime_2015 = pd.read_csv('./data/crime/crime_2015.csv')
crime_2016 = pd.read_csv('./data/crime/crime_2016.csv')
crime_2017 = pd.read_csv('./data/crime/crime_2017.csv')
crime_2018 = pd.read_csv('./data/crime/crime_2018.csv')
crime_2019 = pd.read_csv('./data/crime/crime_2019.csv')
crime_2020 = pd.read_csv('./data/crime/crime_2020.csv')

In [14]:
crime_11_20 = pd.concat([crime_2011, crime_2012, crime_2013, crime_2014, crime_2015, crime_2016, crime_2017, 
                         crime_2018, crime_2019, crime_2020])
crime_11_20 = crime_11_20.drop("Unnamed: 0",axis=1)
max_pop = crime_11_20['state_abb'][crime_11_20['population'].idxmax()]
print(max_pop)
min_pop = crime_11_20['state_abb'][crime_11_20['population'].idxmin()]
print(min_pop)

165120    NY
165120    NY
165120    NY
165120    NY
165120    NY
165120    NY
165120    NY
165120    NJ
165120    NJ
165120    NJ
Name: state_abb, dtype: object
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
Name: state_abb, dtype: object


In [15]:
crime_11_20 = crime_11_20.groupby(['state_abb','year','month']).sum().reset_index()
crime_11_20['crime_proportion'] = crime_11_20['actual_rape_total']/crime_11_20['population']
#crime_11_20 = crime_11_20.drop(columns=['population'])
crime_11_20['state_abb'] = crime_11_20['state_abb'].str.lower()
months = {'january': 1, 'february': 2, 'march': 3, 'april':4, 'may':5, 'june':6, 'july':7, 'august':8, 
          'september':9, 'october':10, 'november':11, 'december':12}
for index, row in crime_11_20.iterrows():
    crime_11_20['month'][index] = months[row['month']]
crime_11_20 = crime_11_20.rename(columns={'state_abb': 'USstate'})
crime_11_20

/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_57593/1061243315.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_11_20['month'][index] = months[row['month']]


,USstate,year,month,population,actual_rape_total,crime_proportion
0,ak,2011,4,725910,26,0.000036
1,ak,2011,8,725910,45,0.000062
2,ak,2011,12,725910,32,0.000044
3,ak,2011,2,725910,29,0.000040
4,ak,2011,1,725910,36,0.000050
...,...,...,...,...,...,...
6475,wy,2020,3,593429,29,0.000049
6476,wy,2020,5,593429,18,0.000030
6477,wy,2020,11,593429,23,0.000039
6478,wy,2020,10,593429,40,0.000067


## 2. Alcohol Data for 2012-2020

In [8]:
alcohol_17 = pd.read_csv('./data/alcohol/alcohol_17.csv')
alcohol_18 = pd.read_csv('./data/alcohol/alcohol_18.csv')
alcohol_19 = pd.read_csv('./data/alcohol/alcohol_19.csv')
alcohol_20 = pd.read_csv('./data/alcohol/alcohol_20.csv')
alcohol_17['YEAR'] = 2017
alcohol_18['YEAR'] = 2018
alcohol_19['YEAR'] = 2019
alcohol_20['YEAR'] = 2020
alcohol_17_20 = pd.concat([alcohol_17, alcohol_18, alcohol_19, alcohol_20])

new_df = alcohol_17_20['Geographic Area Name'].str.split(', ', expand=True)
alcohol_17_20['State_name'] = new_df[1]
alcohol_17_20.drop(columns =["Geographic Area Name"], inplace = True)
alcohol_17_20['alcohol'] = 0
alcohol_17_20 = alcohol_17_20.groupby(['YEAR', 'State_name']).count().reset_index()
alcohol_17_20['avg_alcohol'] = round(alcohol_17_20['alcohol']/12, 2)
alcohol_17_20 = pd.merge(alcohol_17_20, abbr_name, left_on='State_name', right_on='full', how='left')
alcohol_17_20 = alcohol_17_20.dropna(subset=['full'])
alcohol_17_20 = alcohol_17_20.drop(columns=['full', 'alcohol'])

alcohol_12_16 = pd.read_csv('./data/alcohol/alcohol_2012_2016.csv')
alcohol_12_16['avg_alcohol'] = round(alcohol_12_16['alcohol']/12, 2)
alcohol_12_16 = alcohol_12_16.fillna(0)
alcohol_12_16 = pd.merge(alcohol_12_16, code_merged, left_on='cbsa', right_on='CBSA Code', how='left')
alcohol_12_16 = alcohol_12_16.dropna(subset=['abbr'])
alcohol_12_16 = alcohol_12_16.drop(columns=['CBSA Code', 'FIPS State Code', 'FIPS County Code', 'cbsa', 'alcohol'])
alcohol_12_16 = alcohol_12_16.rename(columns={'State Name': 'State_name'})

alcohol_df = pd.concat([alcohol_12_16, alcohol_17_20])
alcohol_df = alcohol_df.groupby(['YEAR','abbr']).sum().reset_index()

# to check whether there is missing data
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for i in range(len(state)):
    al_per_state = alcohol_df[alcohol_df['abbr'] == state[i]]
    exist_year = al_per_state['YEAR'].to_list()
    for j in year:
        if j not in exist_year:
            new_row = {'YEAR': j, 'abbr': state[i]}
            alcohol_df = alcohol_df.append(new_row, ignore_index=True)

# Since it does not data for 'Washington D.C', we take the remaining 50 states for alcohol
print(alcohol_df[alcohol_df['abbr'] == 'al'])
#alcohol_df['YEAR'] = pd.to_datetime(alcohol_df.YEAR, format='%Y')
alcohol_df = alcohol_df[alcohol_df['abbr'] != 'dc']
alcohol_df['YEAR'] = pd.to_numeric(alcohol_df['YEAR'])
alcohol_df = alcohol_df.rename(columns={'abbr': 'USstate', 'YEAR': 'year'})
alcohol_df

     YEAR abbr  avg_alcohol
1    2012   al        86.40
51   2013   al        80.84
101  2014   al        80.66
151  2015   al        85.10
201  2016   al        82.45
251  2017   al         5.58
301  2018   al         5.50
351  2019   al         5.42
401  2020   al         5.50


,year,USstate,avg_alcohol
0,2012,ak,9.25
1,2012,al,86.40
2,2012,ar,82.83
3,2012,az,26.85
4,2012,ca,653.65
...,...,...,...
445,2020,vt,2.08
446,2020,wa,3.75
447,2020,wi,5.00
448,2020,wv,1.42


## 3. Twitter Data 2012-2021

In [9]:
# use proportion of misogyny tweets
all_tweets = pd.read_excel('./data/tweet/All_Tweets_State_Month_Year_05082022.xlsx')
misogyny_tweets = pd.read_excel('./data/tweet/Misogyny_Tweets_State_Month_Year_05082022.xlsx')
tweets = pd.merge(all_tweets, misogyny_tweets, on = ['USstate', 'year', 'month.keyword: Descending'])
tweets['misogyny_proportion'] = tweets['nTweets_y']/tweets['nTweets_x']
#tweets = tweets.drop(columns=['nTweets_x', 'nTweets_y'])
tweets = tweets.sort_values(by=['year', 'month.keyword: Descending'], ascending=True)
tweets = tweets.rename(columns={'month.keyword: Descending': 'month'})

tweets_state = set(tweets['USstate'].to_list())
tweets

,USstate,year,month,nTweets_x,nTweets_y,misogyny_proportion
82,ar,2012,1,7733,1,0.000129
185,ca,2012,1,134228,5,0.000037
338,ct,2012,1,10732,1,0.000093
423,fl,2012,1,70225,3,0.000043
691,il,2012,1,54768,4,0.000073
...,...,...,...,...,...,...
1375,mo,2021,12,34738,1,0.000029
1809,ny,2021,12,226477,2,0.000009
2093,pa,2021,12,82985,1,0.000012
2266,tn,2021,12,47338,1,0.000021


In [10]:
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
full_tweet = pd.DataFrame(columns=['USstate', 'year', 'month'])
for i in range(len(tweets_state)):
    for j in year:
        for k in month:
            full_tweet = full_tweet.append({'USstate': state[i], 'year': j, 'month': k}, ignore_index=True)
print(len(tweets_state))

47


In [11]:
# use spline to fill the missing data
tweets = pd.merge(tweets, full_tweet, on=['USstate', 'year', 'month'], how='right')
tweets['misogyny_proportion'] = tweets['misogyny_proportion'].interpolate(method ='linear', limit_direction ='backward')
tweets['nTweets_x'] = tweets['nTweets_x'].interpolate(method ='linear', limit_direction ='backward')
tweets['nTweets_y'] = tweets['nTweets_y'].interpolate(method ='linear', limit_direction ='backward')
tweets['year'] = pd.to_numeric(tweets['year'])
tweets['month'] = pd.to_numeric(tweets['month'])
tweets.to_csv('./data/tweets_df.csv')
tweets

,USstate,year,month,nTweets_x,nTweets_y,misogyny_proportion
0,al,2012,1,51744.00,3.0,0.000058
1,al,2012,2,51744.00,3.0,0.000058
2,al,2012,3,55089.00,2.0,0.000036
3,al,2012,4,38471.00,1.0,0.000026
4,al,2012,5,36891.00,1.0,0.000027
...,...,...,...,...,...,...
5635,va,2021,8,31318.00,1.0,0.000032
5636,va,2021,9,31468.75,1.0,0.000032
5637,va,2021,10,31619.50,1.0,0.000032
5638,va,2021,11,31770.25,1.0,0.000031


# 4. Merge all datasets

In [16]:
# alcohol and twitter
al_twitter = pd.merge(tweets, alcohol_df, right_on=['USstate','year'], left_on=['USstate', 'year'], how='left')
al_twitter = al_twitter.dropna()

# add crime
merged_df = pd.merge(al_twitter, crime_11_20, left_on=['USstate', 'year', 'month'], right_on=['USstate', 'year', 'month'], how='left')
merged_df['crime_proportion'] = merged_df['crime_proportion'].interpolate(method ='linear', limit_direction ='backward')
merged_df['actual_rape_total'] = merged_df['actual_rape_total'].interpolate(method ='linear', limit_direction ='backward')
merged_df

,USstate,year,month,nTweets_x,nTweets_y,misogyny_proportion,avg_alcohol,population,actual_rape_total,crime_proportion
0,al,2012,1,51744.00,3.0,0.000058,86.40,4834334,106,0.000022
1,al,2012,2,51744.00,3.0,0.000058,86.40,4834334,70,0.000014
2,al,2012,3,55089.00,2.0,0.000036,86.40,4834334,78,0.000016
3,al,2012,4,38471.00,1.0,0.000026,86.40,4834334,86,0.000018
4,al,2012,5,36891.00,1.0,0.000027,86.40,4834334,101,0.000021
...,...,...,...,...,...,...,...,...,...,...
4963,va,2020,8,85152.00,2.0,0.000023,8.42,8600924,236,0.000027
4964,va,2020,9,86418.50,1.5,0.000017,8.42,8600924,193,0.000022
4965,va,2020,10,87685.00,1.0,0.000011,8.42,8600924,198,0.000023
4966,va,2020,11,74017.25,1.0,0.000016,8.42,8600924,166,0.000019


In [17]:
merged_df.to_csv('./data/merged_data.csv')